In [10]:
import requests
import pandas as pd

link = "https://olinda.bcb.gov.br/olinda/servico/mecir_dinheiro_em_circulacao/versao/v1/odata/informacoes_diarias?$top=10000&$orderby=Data%20desc&$format=json"

requisicao = requests.get(link)
informacoes = requisicao.json()

In [11]:
tabela = pd.DataFrame(informacoes["value"])

display(tabela)

,Data,Quantidade,Valor,Denominacao,Especie
0,2024-05-07,3191004929,3.191005e+07,0.01,Moedas
1,2024-05-07,7961617243,3.980809e+08,0.05,Moedas
2,2024-05-07,8122037414,8.122037e+08,0.10,Moedas
3,2024-05-07,3652636300,9.131591e+08,0.25,Moedas
4,2024-05-07,3684390728,1.842195e+09,0.50,Moedas
...,...,...,...,...,...
9995,2021-07-02,72938286,1.458766e+10,200.00,Cédulas
9996,2021-07-01,3191178998,3.191179e+07,0.01,Moedas
9997,2021-07-01,7207631645,3.603816e+08,0.05,Moedas
9998,2021-07-01,7430976946,7.430977e+08,0.10,Moedas


In [12]:
# pegar todas as informacoes
tabela_final = pd.DataFrame()
pular_indice = 0

while True:
    link = f'https://olinda.bcb.gov.br/olinda/servico/mecir_dinheiro_em_circulacao/versao/v1/odata/informacoes_diarias?$top=10000&$skip={pular_indice}&$orderby=Data%20desc&$format=json'
    requisicao = requests.get(link)
    informacoes = requisicao.json()
    tabela = pd.DataFrame(informacoes["value"])
    if len(informacoes['value']) < 1:
        break
    tabela_final = pd.concat([tabela_final, tabela])
    pular_indice += 10000

display(tabela_final)

,Data,Quantidade,Valor,Denominacao,Especie
0,2024-05-07,3191004929,3.191005e+07,0.01,Moedas
1,2024-05-07,7961617243,3.980809e+08,0.05,Moedas
2,2024-05-07,8122037414,8.122037e+08,0.10,Moedas
3,2024-05-07,3652636300,9.131591e+08,0.25,Moedas
4,2024-05-07,3684390728,1.842195e+09,0.50,Moedas
...,...,...,...,...,...
3730,1994-10-03,181609358,1.816094e+08,1.00,Moedas
3731,1994-10-03,252922174,1.264611e+09,5.00,Cédulas
3732,1994-10-03,273630983,2.736310e+09,10.00,Cédulas
3733,1994-10-03,28945486,1.447274e+09,50.00,Cédulas


In [17]:
# Salvar os dados em um arquivo CSV
tabela_final.to_csv('dados.csv', index=False)

In [13]:
#Tendência de Uso de Moedas versus Cédulas ao Longo do Tempo
def obter_dados():
    link_base = "https://olinda.bcb.gov.br/olinda/servico/mecir_dinheiro_em_circulacao/versao/v1/odata/informacoes_diarias?$top=10000&$orderby=Data%20desc&$format=json"
    tabela_final = pd.DataFrame()
    pular_indice = 0

    while True:
        link = f'{link_base}&$skip={pular_indice}'
        requisicao = requests.get(link)
        informacoes = requisicao.json()
        tabela = pd.DataFrame(informacoes["value"])
        if len(informacoes['value']) < 1:
            break
        tabela_final = pd.concat([tabela_final, tabela])
        pular_indice += 10000

    return tabela_final

def tendencia_uso_moedas_cedulas(df):
    df['Data'] = pd.to_datetime(df['Data'])  
    df['Ano'] = df['Data'].dt.year 
    df_agrupado = df.groupby(['Ano', 'Especie']).agg({'Quantidade': 'sum'}).reset_index()
    pivot = df_agrupado.pivot(index='Ano', columns='Especie', values='Quantidade').fillna(0)
    pivot['Total'] = pivot['Cédulas'] + pivot['Moedas']
    pivot['% Moedas'] = pivot['Moedas'] / pivot['Total'] * 100
    pivot['% Cédulas'] = pivot['Cédulas'] / pivot['Total'] * 100

    return pivot

dados = obter_dados()
insight_tendencia = tendencia_uso_moedas_cedulas(dados)
print(insight_tendencia)

Especie        Cédulas         Moedas          Total   % Moedas  % Cédulas
Ano                                                                       
1994       53606226142   148753135308   202359361450  73.509392  26.490608
1995      205662829391   842750925954  1048413755345  80.383429  19.616571
1996      244780471688  1016844391027  1261624862715  80.597999  19.402001
1997      292023409857  1187671163922  1479694573779  80.264616  19.735384
1998      318542674333  1344989299962  1663531974295  80.851425  19.148575
1999      355958880408  1511945859087  1867904739495  80.943414  19.056586
2000      414532945775  1642894485465  2057427431240  79.851880  20.148120
2001      473657861975  1720815051189  2194472913164  78.415871  21.584129
2002      585818021880  1920252536450  2506070558330  76.624041  23.375959
2003      615275538533  2120783573606  2736059112139  77.512345  22.487655
2004      678383146258  2339903078382  3018286224640  77.524227  22.475773
2005      730203889934  2

In [14]:
#Distribuição de Cédulas e Moedas por Denominação
def distribuicao_por_denominacao(df):
    df_agrupado = df.groupby(['Denominacao', 'Especie']).agg({'Quantidade': 'sum'}).reset_index()
    pivot = df_agrupado.pivot(index='Denominacao', columns='Especie', values='Quantidade').fillna(0)

    return pivot

insight_distribuicao_denominacao = distribuicao_por_denominacao(dados)
print(insight_distribuicao_denominacao)

Especie           Cédulas        Moedas
Denominacao                            
0.01         0.000000e+00  2.084558e+13
0.05         0.000000e+00  2.892694e+13
0.10         0.000000e+00  3.073979e+13
0.25         0.000000e+00  1.294665e+13
0.50         0.000000e+00  1.241592e+13
1.00         2.434641e+12  1.275102e+13
10.00        4.391925e+12  0.000000e+00
100.00       3.835952e+12  0.000000e+00
2.00         4.963918e+12  0.000000e+00
20.00        3.019451e+12  0.000000e+00
200.00       9.058422e+10  0.000000e+00
5.00         2.815677e+12  0.000000e+00
50.00        8.890367e+12  0.000000e+00


In [15]:
#Flutuações de Quantidade de Cédulas e Moedas por Espécie ao Longo do Tempo
def flutuacoes_quantidade_por_especie(df):
    df['Data'] = pd.to_datetime(df['Data'])
    df_agrupado = df.groupby(['Data', 'Especie']).agg({'Quantidade': 'sum'}).reset_index()
    pivot = df_agrupado.pivot(index='Data', columns='Especie', values='Quantidade').fillna(0)

    return pivot

insight_flutuacoes_quantidade = flutuacoes_quantidade_por_especie(dados)
print(insight_flutuacoes_quantidade)

Especie        Cédulas       Moedas
Data                               
1994-10-03   826194330   2021542673
1994-10-04   823730036   2024082074
1994-10-05   808897373   2038182968
1994-10-06   850331678   2053793172
1994-10-07   904876649   2070464842
...                ...          ...
2024-04-30  7453940490  30825079058
2024-05-02  7478326915  30829767058
2024-05-03  7492191401  30833966558
2024-05-06  7509152451  30835555558
2024-05-07  7517006294  30836795010

[7427 rows x 2 columns]
